In [35]:
import numpy as np
import cv2 as cv
from PIL import Image
import matplotlib.pyplot as plt

img1 = cv.imread("checker9.jpg")
#img1 = cv.resize(img1, (960, 540))      
gray1 = cv.cvtColor(img1,cv.COLOR_BGR2GRAY)

criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)

ret, corners = cv.findChessboardCorners(gray1, (7,10), cv.CALIB_CB_ADAPTIVE_THRESH + cv.CALIB_CB_FAST_CHECK + cv.CALIB_CB_NORMALIZE_IMAGE + cv.CALIB_USE_INTRINSIC_GUESS)

if ret == True:
    corners2 = cv.cornerSubPix(gray1,corners, (11,11), (-1,-1),criteria)
     # Draw and display the corners
    cv.drawChessboardCorners(img1, (7,10), corners2, ret)
    cv.imshow('img', img1)
    cv.waitKey(0)
    cv.destroyAllWindows()
else:
    print("Not Found")


error: OpenCV(4.6.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'
